# Tailored Tutorial

Tutorial
 
https://medium.com/@naresh.kancharla/pydantic-ai-ai-agent-library-021e7983d4c8

Ollama Local usage documentation

 https://ai.pydantic.dev/api/models/ollama


In [1]:
!uv pip install pydantic-ai
!uv pip install pydantic-ai-slim 



Using Python 3.11.10 environment at /Users/rianders/Documents/notebooks/.venv
Audited 1 package in 18ms
Using Python 3.11.10 environment at /Users/rianders/Documents/notebooks/.venv
Audited 1 package in 2ms


In [2]:
# notebooks % ollama run llama3.2-vision:latest
from pydantic_ai import Agent
import nest_asyncio
import asyncio

from pprint import pprint


In [3]:
# llama3.2-vision:latest doesn't support tool calling
# llama3.2 should support tool calling



# Apply nest_asyncio first
nest_asyncio.apply()

ask_agent = Agent(  
    'ollama:llama3.2:3b',
    #system_prompt='Be concise, reply with one sentence.',  
)

result = ask_agent.run_sync('Where were the olympics held in 2012 ?')  
print(result.data)

The 2012 Summer Olympics took place in London, United Kingdom. Specifically, the events were held across various venues in London, including:

* Olympic Stadium
* Wembley Stadium
* Hampden Park (Scotland)
* Lord's Cricket Ground
* Riverbank Centre for Performing Arts
* Alexandra Palace
* London Stadium

The Paralympic Games, which took place from September 29 to October 9, 2012, were also held in London.


In [4]:
from pydantic import BaseModel


In [5]:
class CityLocation(BaseModel):
    city: str
    country: str

city_country_agent = Agent('ollama:llama3.2:3b', 
                       result_type=CityLocation)



In [6]:
result = city_country_agent.run_sync('Which city and country were the olympics held in 2012?')
print(result.data)

city='London' country='United Kingdom'


In [9]:
result = ask_agent.run_sync('Where is Fair use Building and Research Labs or FUBAR Labs maker space located?')
print(result.data)
result = city_country_agent.run_sync('Where is Fair use Building and Research Labsor FUBAR Labs  maker space located?')
print(result.data)

I couldn't find any information on "Fair use Building and Research Labs" or "FUBAR Labs". However, I did find information on the "FUBAR Labs", which is a makerspace with multiple locations. 

There are a few places that go by FUBAR labs: there's one in Kansas City, Missouri, USA; another in Denver Colorado; and also Seattle Washington
city='Fair use' country='Building and Research Labsor Fubar labs  maker space: FUBAR Labs'


In [10]:
coordinates = "40°27'33\"N 74°29'02\"W"

result = ask_agent.run_sync(f'What country and city are these coordinates in: {coordinates}?')
print(result.data)




The coordinates 40°27'33"N 74°29'02"W are located in the United States. However, I cannot determine which city they are associated with without additional information.


In [11]:
result = city_country_agent.run_sync(f'What country and city are these coordinates in:  {coordinates}?')
print(result.data)
print(result.usage())

UnexpectedModelBehavior: Exceeded maximum retries (1) for result validation

In [12]:
# Let's update the agent with tools

location_agent = Agent('ollama:llama3.2:3b', 
                       system_prompt=(
                           """You are a specialized assistant that converts natural language location queries into structured city and country information. Use the provided tools (point_to_latlong, coords_to_address, address_to_location, geocode_with_retry) to process location queries, choosing point_to_latlong for coordinate strings and address_to_location for place names, landmarks, or addresses. Always return results in CityLocation format with required city and country fields, including state and coordinates when available, defaulting to the most prominent location when ambiguous (e.g., 'Paris' should return Paris, France)."""
                       ),
                       result_type=CityLocation)

In [13]:
# Let's update the agent with tools
result = location_agent.run_sync(f'What country and city is the statue of liberty located in?')
print(result.data)
print(result.usage())


UnexpectedModelBehavior: Exceeded maximum retries (1) for result validation

In [ ]:
# https://geopy.readthedocs.io/en/latest/

In [ ]:
!uv pip install geopy

In [ ]:
from geopy.geocoders import Nominatim
from geopy.point import Point
from typing import Optional, Tuple

from pydantic_ai import Agent, RunContext, Tool



In [ ]:
# https://ai.pydantic.dev/tools/
# The tools have to be attached to the correct agent or the agent to use it.

@location_agent.tool_plain
def coords_to_address(coordinates):
    """Convert coordinates string or Point object to address"""
    geolocator = Nominatim(user_agent="my_app")
    
    # If input is a string, convert to Point
    if isinstance(coordinates, str):
        point = Point(coordinates)
    else:
        point = coordinates
        
    location = geolocator.reverse(f"{point.latitude}, {point.longitude}")
    return location.address

@location_agent.tool_plain
def address_to_coords(address):
    """Convert address to coordinates, returns Point object"""
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode(address)
    
    if location:
        return Point(location.latitude, location.longitude)
    return None



In [ ]:
@location_agent.tool_plain
def latlong_to_address(latitude: float, longitude: float) -> Optional[str]:
    """
    Convert latitude and longitude to address
    
    Args:
        latitude (float): Latitude in decimal degrees
        longitude (float): Longitude in decimal degrees
    
    Returns:
        Optional[str]: Address string or None if not found
    """
    geolocator = Nominatim(user_agent="my_app")
    try:
        location = geolocator.reverse(f"{latitude}, {longitude}")
        if location:
            return str(location.address)
        return None
    except Exception as e:
        print(f"Error getting address: {e}")
        return None


In [ ]:
@location_agent.tool_plain
def point_to_latlong(point_str: str) -> Tuple[float, float]:
    """
    Convert a coordinate string to latitude and longitude
    
    Args:
        point_str (str): Coordinate string (e.g. "40°27'33"N 74°29'02"W")
    
    Returns:
        Tuple[float, float]: Tuple containing (latitude, longitude)
    """
    point = Point(point_str)
    return float(point.latitude), float(point.longitude)

In [ ]:
coords = "40°27'33\"N 74°29'02\"W"
point = Point(coords)
lat, lon = point_to_latlong(point)
print(f"Latitude: {lat}")
print(f"Longitude: {lon}")

In [ ]:
latlong_to_address(lat,lon) 


In [ ]:

# Address to coordinates
address = "1510b Jersey Ave, NJ 08902"
point = address_to_coords(address)
if point:
    print(f"Coordinates: {point.latitude}°N, {point.longitude}°E")

In [ ]:
# Example usage:
# Coordinates to address
coordinates = "40°27'33\"N 74°29'02\"W"
"""
40°27'33"N 74°29'02"W
"""
address = coords_to_address(coordinates)
print(f"Address: {address}")



In [ ]:
loc_result = location_agent.run_sync('Find the city and state Rutgers University')  
print(loc_result.data)

In [ ]:
pprint(loc_result.all_messages(), indent=2, width=80)
